In [1]:
%cd ..

/fs01/home/abbasgln/codes/medAI/projects/tta


/h/abbasgln/.conda/envs/mttt/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from dotenv import load_dotenv
# Loading environment variables
load_dotenv()

import torch
import torch.nn as nn
import torch.nn.functional as F
import typing as tp
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from dataclasses import dataclass, field
import logging
import wandb

import medAI
from medAI.utils.setup import BasicExperiment, BasicExperimentConfig

from utils.metrics import MetricCalculator

from timm.optim.optim_factory import create_optimizer

from einops import rearrange, repeat
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import timm

from copy import copy, deepcopy
import pandas as pd

from datasets.datasets import ExactNCT2013RFImagePatches
from medAI.datasets.nct2013 import ExactNCT2013BmodePatches
from medAI.datasets.nct2013 import (
    KFoldCohortSelectionOptions,
    LeaveOneCenterOutCohortSelectionOptions, 
    PatchOptions
)
import torchmetrics



In [3]:
LEAVE_OUT='CRCEO'

In [4]:
from ensemble_experiment import EnsembleConfig
config = EnsembleConfig(cohort_selection_config=LeaveOneCenterOutCohortSelectionOptions(leave_out=f"{LEAVE_OUT}"),
                        # patch_config=PatchOptions(needle_mask_threshold=0.6, prostate_mask_threshold=0.9, patch_size_mm = (3,3), strides = (1.2,1.2))
)

from baseline_experiment import BaselineConfig
from torchvision.transforms import v2 as T
from torchvision.tv_tensors import Image as TVImage

class Transform:
    def __init__(selfT, augment=False):
        selfT.augment = augment
        selfT.size = (256, 256)
        # Augmentation
        selfT.transform = T.Compose([
            T.RandomAffine(degrees=0, translate=(0.2, 0.2)),
            T.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0.5),
            T.RandomHorizontalFlip(p=0.5),
            T.RandomVerticalFlip(p=0.5),
        ])  
    
    def __call__(selfT, item):
        patch = item.pop("patch")
        patch = copy(patch)
        patch = (patch - patch.min()) / (patch.max() - patch.min()) \
            if config.instance_norm else patch
        patch = TVImage(patch)
        patch_orig_size = patch
        patch = T.Resize(selfT.size, antialias=True)(patch).float()
        
        label = torch.tensor(item["grade"] != "Benign").long()
        
        if selfT.augment:
            patch_augs = torch.stack([selfT.transform(patch) for _ in range(2)], dim=0)
            return patch_augs, patch, label, item #, patch_orig_size
        
        return -1, patch, label, item

if (config.cohort_selection_config.leave_out == "UVA"):
    config.cohort_selection_config.benign_to_cancer_ratio = 1.5 # 5.0    
    config.patch_config = PatchOptions(needle_mask_threshold=-1, prostate_mask_threshold=0.9, patch_size_mm = (5,5), strides = (1,1))

if (config.cohort_selection_config.leave_out == "CRCEO"):
    config.cohort_selection_config.benign_to_cancer_ratio = 1.5    
    config.patch_config = PatchOptions(needle_mask_threshold=-1, prostate_mask_threshold=0.9, patch_size_mm = (5,5), strides = (1,1))

test_ds_memo = ExactNCT2013RFImagePatches(
    split="test",
    transform=Transform(augment=True),
    cohort_selection_options=config.cohort_selection_config,
    patch_options=config.patch_config,
    # patch_options=patch_config,
    debug=config.debug,
)

test_ds_bmode = ExactNCT2013BmodePatches(
    split="test",
    transform=Transform(augment=True),
    cohort_selection_options=config.cohort_selection_config,
    patch_options=config.patch_config,
    # patch_options=patch_config,
)

test_loader_memo = DataLoader(
    test_ds_memo, batch_size=32, shuffle=False, num_workers=4
)

test_core_ds = test_ds_bmode.dataset.dataset

Computing positions: 100%|██████████| 535/535 [00:35<00:00, 15.00it/s]


## UP-Label

In [5]:
from baseline_experiment import FeatureExtractorConfig

fe_config = FeatureExtractorConfig()

# Create the model
list_models: tp.List[nn.Module] = [timm.create_model(
    fe_config.model_name,
    num_classes=fe_config.num_classes,
    in_chans=1,
    features_only=fe_config.features_only,
    norm_layer=lambda channels: nn.GroupNorm(
                    num_groups=fe_config.num_groups,
                    num_channels=channels
                    )) for _ in range(5)]

# list_models: tp.List[nn.Module] = [timm.create_model(
#     fe_config.model_name,
#     num_classes=fe_config.num_classes,
#     in_chans=1,
#     features_only=fe_config.features_only,
#     ) for _ in range(5)]

CHECkPOINT_PATH = os.path.join(os.getcwd(), f'logs/tta/ensemble_5mdls_gn_3ratio_loco/ensemble_5mdls_gn_3ratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')
# CHECkPOINT_PATH = os.path.join(os.getcwd(), f'logs/tta/ensemble_5mdls_gn_avgprob_3ratio_loco/ensemble_5mdls_gn_avgprob_3ratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')
# CHECkPOINT_PATH = os.path.join(os.getcwd(), f'logs/tta/ensemble_5mdls_bn_1nratio_loco/ensemble_5mdls_bn_1nratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')
# CHECkPOINT_PATH = os.path.join(os.getcwd(), f'logs/tta/ensemble_5mdls_gn_1ratio_loco/ensemble_5mdls_gn_1ratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')

state = torch.load(CHECkPOINT_PATH)
[model.load_state_dict(state["list_models"][i]) for i, model in enumerate(list_models)]

[model.eval() for model in list_models]
[model.cuda() for model in list_models]

a = True

In [6]:
temp = 1.0
beta = 0.0
if LEAVE_OUT == "JH":
#     temp = 1.6793
#     beta = -1.0168
    temp = 0.9253
    beta = -1.0273
elif LEAVE_OUT == "PCC":
#     temp = 1.5950
#     beta = -0.8514
    temp = 1.0075
    beta = -0.8614
elif LEAVE_OUT == "PMCC":
#     temp = 0.6312
#     beta = -1.0017
    temp = 0.9020
    beta = -1.0609
elif LEAVE_OUT == "UVA":
#     temp = 0.9333
#     beta = -0.7474
    temp = 1.6528
    beta = -0.6192
elif LEAVE_OUT == "CRCEO":
#     temp = 1.2787
#     beta = -0.8716
    temp = 0.8515
    beta = -0.8461
    
temp = torch.tensor(temp).cuda()
beta = torch.tensor(beta).cuda()

In [7]:
loader = test_loader_memo
enable_pseudo_label = True
temp_scale = False
certain_threshold = 0.8
thr = 0.4

metric_calculator = MetricCalculator()
desc = "test"

for i, batch in enumerate(tqdm(loader, desc=desc)):
    images_augs, images, labels, meta_data = batch
    # images_augs = images_augs.cuda()
    images = images.cuda()
    labels = labels.cuda()
    
    adaptation_model_list = [deepcopy(model) for model in list_models] 
    [model.eval() for model in adaptation_model_list]

    
    if enable_pseudo_label:
        params = []
        for model in adaptation_model_list:
            params.append({'params': model.parameters()})
        optimizer = optim.SGD(params, lr=5e-4)
        
        # Adapt to test
        for j in range(1):
            optimizer.zero_grad()
            
            # Forward pass
            stacked_logits = torch.stack([model(images) for model in adaptation_model_list])
            if temp_scale:
                stacked_logits = stacked_logits / temp + beta
            
            # Remove uncertain samples from test-time adaptation
            avg_probs = F.softmax(stacked_logits, dim=-1).mean(dim=0)
            certain_idx = F.softmax(stacked_logits, dim=-1).mean(dim=0).max(dim=-1)[0] >= certain_threshold
            # certain_idx =  torch.sum((-avg_probs*torch.log(avg_probs)), dim=-1) <= certain_threshold
            stacked_logits = stacked_logits[:, certain_idx, ...]
            
            list_losses = []
            for k, outputs in enumerate(adaptation_model_list):
                loss = nn.CrossEntropyLoss()(stacked_logits[k, ...], (F.softmax(stacked_logits, dim=-1).mean(dim=0)[:, 1] >= thr).to(torch.long))
                list_losses.append(loss.mean())
            # Backward pass
            sum(list_losses).backward()
            optimizer.step()
        
    # Evaluate
    logits = torch.stack([model(images) for model in adaptation_model_list])
    if temp_scale:
        logits = logits / temp + beta
    losses = [nn.CrossEntropyLoss()(
        logits[i, ...],
        labels
        ) for i in range(5)
    ]
                    
    # Update metrics   
    metric_calculator.update(
        batch_meta_data = meta_data,
        probs = F.softmax(logits, dim=-1).mean(dim=0).detach().cpu(), # Take mean over ensembles
        labels = labels.detach().cpu(),
        raw_probs = F.softmax(logits, dim=-1).detach().cpu(),
    )
    
METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/ensemble_5mdls_gn_3ratio_loco/ensemble_5mdls_gn_3ratio_loco_{LEAVE_OUT}/metric_calculator_prst.pth'

torch.save(metric_calculator, METRIC_CALCULATOR_PATH)

test:   0%|          | 0/6065 [00:00<?, ?it/s]

## MEMO and Baseline GN and BN

In [5]:
## Model
from vicreg_pretrain_experiment import TimmFeatureExtractorWrapper
from timm.layers.adaptive_avgmax_pool import SelectAdaptivePool2d
fe_config = config.model_config
# Create the model
model: nn.Module = timm.create_model(
    fe_config.model_name,
    num_classes=fe_config.num_classes,
    in_chans=1,
    features_only=fe_config.features_only,
    norm_layer=lambda channels: nn.GroupNorm(
                    num_groups=fe_config.num_groups,
                    num_channels=channels
                    ))

# model: nn.Module = timm.create_model(
#     fe_config.model_name,
#     num_classes=fe_config.num_classes,
#     in_chans=1,
#     features_only=fe_config.features_only)

CHECkPOINT_PATH = os.path.join(f'/fs01/home/abbasgln/codes/medAI/projects/tta/logs/tta/baseline_gn_crtd3ratio_loco/baseline_gn_crtd3ratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')
# CHECkPOINT_PATH = os.path.join(f'/fs01/home/abbasgln/codes/medAI/projects/tta/logs/tta/baseline_gn_avgprob_3ratio_loco/baseline_gn_avgprob_3ratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')
# CHECkPOINT_PATH = os.path.join(f'/fs01/home/abbasgln/codes/medAI/projects/tta/logs/tta/baseline_gn_1nratio_loco/baseline_gn_1nratio_loco_{LEAVE_OUT}/', 'best_model.ckpt')

# CHECkPOINT_PATH = os.path.join(f'/fs01/home/abbasgln/codes/medAI/projects/tta/logs/tta/baseline_bn_avgprob_3ratio_loco2/baseline_bn_avgprob_3ratio_loco2_{LEAVE_OUT}/', 'best_model.ckpt')

model.load_state_dict(torch.load(CHECkPOINT_PATH)['model'])
model.eval()
model.cuda()
a = True

In [6]:
## MEMO
loader = test_loader_memo
enable_memo = True

from memo_experiment import batched_marginal_entropy
metric_calculator = MetricCalculator()
desc = "test"

criterion = nn.CrossEntropyLoss()
for i, batch in enumerate(tqdm(loader, desc=desc)):
    batch = deepcopy(batch)
    images_augs, images, labels, meta_data = batch
    images_augs = images_augs.cuda()
    images = images.cuda()
    labels = labels.cuda()
    
    batch_size, aug_size= images_augs.shape[0], images_augs.shape[1]

    # Adapt to test
    _images_augs = images_augs.reshape(-1, *images_augs.shape[2:]).cuda()
    adaptation_model = deepcopy(model)
    adaptation_model.eval()
    if enable_memo:
        optimizer = optim.SGD(adaptation_model.parameters(), lr=5e-4)
        
        for j in range(1):
            outputs = adaptation_model(_images_augs).reshape(batch_size, aug_size, -1)  
            loss, logits = batched_marginal_entropy(outputs)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
    
    # Evaluate
    logits = adaptation_model(images)
    loss = criterion(logits, labels)
                    
    # Update metrics   
    metric_calculator.update(
        batch_meta_data = meta_data,
        probs = nn.functional.softmax(logits, dim=-1).detach().cpu(),
        labels = labels.detach().cpu(),
    )
    
# METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/baseline_gn_crtd3ratio_loco/baseline_gn_crtd3ratio_loco_{LEAVE_OUT}/metric_calculator_prst.pth'
METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/baseline_gn_crtd3ratio_loco/baseline_gn_crtd3ratio_loco_{LEAVE_OUT}/metric_calculator_memo_prst.pth'

torch.save(metric_calculator, METRIC_CALCULATOR_PATH)

test:   0%|          | 0/6065 [00:00<?, ?it/s]

In [7]:
avg_core_probs_first = True
metric_calculator.avg_core_probs_first = avg_core_probs_first

# Log metrics every epoch
metrics = metric_calculator.get_metrics()

# Update best score
(best_score_updated,best_score) = metric_calculator.update_best_score(metrics, desc)

best_score_updated = copy(best_score_updated)
best_score = copy(best_score)
        
# Log metrics
metrics_dict = {
    f"{desc}/{key}": value for key, value in metrics.items()
    }

metrics_dict

{'test/patch_auroc': tensor(0.5946),
 'test/patch_accuracy': tensor(0.5658),
 'test/patch_sensitivity': tensor(0.3763),
 'test/patch_specificity': tensor(0.7554),
 'test/all_inv_patch_auroc': tensor(0.5618),
 'test/all_inv_patch_accuracy': tensor(0.5435),
 'test/all_inv_patch_sensitivity': tensor(0.3316),
 'test/all_inv_patch_specificity': tensor(0.7554),
 'test/core_auroc': tensor(0.8091),
 'test/core_accuracy': tensor(0.7105),
 'test/core_sensitivity': tensor(0.5370),
 'test/core_specificity': tensor(0.8840),
 'test/all_inv_core_auroc': tensor(0.7225),
 'test/all_inv_core_accuracy': tensor(0.6546),
 'test/all_inv_core_sensitivity': tensor(0.4252),
 'test/all_inv_core_specificity': tensor(0.8840)}

## Heatmaps

In [15]:
# UP-Label 
METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/ensemble_5mdls_gn_3ratio_loco/ensemble_5mdls_gn_3ratio_loco_{LEAVE_OUT}/metric_calculator_prst.pth'

UP_Label_metric_calculator = torch.load(METRIC_CALCULATOR_PATH)

In [16]:
# MEMO
METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/baseline_gn_crtd3ratio_loco/baseline_gn_crtd3ratio_loco_{LEAVE_OUT}/metric_calculator_memo_prst.pth'

MEMO_metric_calculator = torch.load(METRIC_CALCULATOR_PATH)

In [17]:
# Baseline BN
# METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/baseline_bn_avgprob_3ratio_loco2/baseline_bn_avgprob_3ratio_loco2_{LEAVE_OUT}/metric_calculator_memo_prst.pth'
METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/baseline_bn_avgprob_3ratio_loco2/baseline_bn_avgprob_3ratio_loco2_{LEAVE_OUT}/metric_calculator_prst.pth'

Baseline_metric_calculator = torch.load(METRIC_CALCULATOR_PATH)

In [18]:
# VICREG finetune
# METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/vicreg_1024-300finetune_1e-4lr_avgprob_gn_crtd3ratio_loco/vicreg_1024-300finetune_1e-4lr_avgprob_gn_crtd3ratio_loco_{LEAVE_OUT}/metric_calculator_memo_prst.pth'
METRIC_CALCULATOR_PATH = f'/ssd005/projects/exactvu_pca/checkpoint_store/Mahdi/vicreg_1024-150finetune_1e-4lr_avgprob_bn_3ratio_loco/vicreg_1024-150finetune_1e-4lr_avgprob_bn_3ratio_loco_{LEAVE_OUT}/metric_calculator_prst.pth'

VICReg_finetune_metric_calculator = torch.load(METRIC_CALCULATOR_PATH)

In [ ]:
test_core_ds[126]

In [6]:
from medAI.datasets.nct2013 import convert_physical_coordinate_to_pixel_coordinate

image = test_core_ds[126]["bmode"]
positions = test_ds_bmode.positions[126]
core_id = test_core_ds[126]['id']
_needle_mask = torch.tensor(test_core_ds[126]["needle_mask"])
needle_alpha = 0.0
needle_rgb_color = np.array([1, 1, 0]).reshape(1, 1, 3)
patch_alpha = 0.2
threshold = 0.6


H, W = image.shape
image = (image - image.min())/image.max()
image = torch.tensor(image).repeat(3, 1, 1).numpy().transpose(1, 2, 0)
_needle_mask = T.Resize((H, W), interpolation=T.InterpolationMode.NEAREST)(_needle_mask.unsqueeze(0)).permute(1, 2, 0).repeat(1,1,3).numpy()
needle_heatmap = (needle_alpha*_needle_mask*needle_rgb_color)

positions = [pos['position'] for pos in positions]
probs = torch.stack(metric_calculator.core_id_probs[core_id])
pred_labels = probs[:, 1] >= threshold

prediction_heatmap = np.zeros_like(image)
for pos, pred in zip(positions, pred_labels):
    xmin_mm, ymin_mm, xmax_mm, ymax_mm= pos
    xmin_px, ymin_px = convert_physical_coordinate_to_pixel_coordinate(
        (xmin_mm, ymin_mm), (28, 46.06), (H,W))
    xmax_px, ymax_px = convert_physical_coordinate_to_pixel_coordinate(
        (xmax_mm, ymax_mm), (28, 46.06), (H, W))
    
    prediction_heatmap[xmin_px:xmax_px, ymin_px:ymax_px, :] += np.array([1, 0, 0]).reshape(1, 1, 3) if pred else np.array([0, 0, 1]).reshape(1, 1, 3)
prediction_heatmap = patch_alpha * prediction_heatmap

heatmap = image + needle_heatmap + prediction_heatmap 



In [21]:
from medAI.datasets.nct2013 import convert_physical_coordinate_to_pixel_coordinate

def create_heatmap(image, positions, needle_mask, core_id, core_id_probs, threshold=0.6, alpha=0.2, needle_alpha=0.0):
    H, W = image.shape
    image = (image - image.min())/image.max()
    image = torch.tensor(image).repeat(3, 1, 1).numpy().transpose(1, 2, 0)
    needle_mask = T.Resize((H, W), interpolation=T.InterpolationMode.NEAREST)(needle_mask.unsqueeze(0)).permute(1, 2, 0).repeat(1,1,3).numpy()
    needle_heatmap = (needle_alpha*needle_mask*np.array([1, 1, 0]).reshape(1, 1, 3))
    probs = torch.stack(core_id_probs[core_id])
    pred_labels = probs[:, 1] >= threshold

    positions = [pos['position'] for pos in positions]
    prediction_heatmap = np.zeros_like(image)
    for pos, pred in zip(positions, pred_labels):
        xmin_mm, ymin_mm, xmax_mm, ymax_mm = pos
        xmin_px, ymin_px = convert_physical_coordinate_to_pixel_coordinate(
            (xmin_mm, ymin_mm), (28, 46.06), (H,W))
        xmax_px, ymax_px = convert_physical_coordinate_to_pixel_coordinate(
            (xmax_mm, ymax_mm),(28, 46.06), (H, W))
        prediction_heatmap[xmin_px:xmax_px, ymin_px:ymax_px, :] += np.array([1, 0, 0]).reshape(1, 1, 3) if pred else np.array([0, 0, 1]).reshape(1, 1, 3)
    prediction_heatmap = alpha * prediction_heatmap

    heatmap = image + needle_heatmap + prediction_heatmap
    return heatmap

In [192]:
core_no = 210
threshold = 0.3
alpha = 0.02

bmode_image = test_core_ds[core_no]["bmode"]
positions = test_ds_bmode.positions[core_no]
needle_mask = torch.tensor(test_core_ds[core_no]["needle_mask"])
core_id = test_core_ds[core_no]['id']
involvement = test_core_ds[core_no]["pct_cancer"]

patch_probs = Baseline_metric_calculator.core_id_probs
Baseline_heatmap = create_heatmap(bmode_image, positions, needle_mask, core_id, patch_probs, threshold=0.3, alpha=alpha, needle_alpha=0.1)

patch_probs = VICReg_finetune_metric_calculator.core_id_probs
VICReg_heatmap = create_heatmap(bmode_image, positions, needle_mask, core_id, patch_probs, threshold=0.3, alpha=alpha, needle_alpha=0.1)

patch_probs = MEMO_metric_calculator.core_id_probs
MEMO_heatmap = create_heatmap(bmode_image, positions, needle_mask, core_id, patch_probs, threshold=0.3, alpha=alpha, needle_alpha=0.1)

patch_probs = UP_Label_metric_calculator.core_id_probs
UP_Label_heatmap = create_heatmap(bmode_image, positions, needle_mask, core_id, patch_probs, threshold=0.5, alpha=alpha, needle_alpha=0.1)

print(f"Core No: {core_no}, Inv: {involvement:.2f}")

plt.figure(figsize=(16, 6))  # Set the figure size (width, height) in inches

# First subplot
plt.subplot(1, 4, 1) # (rows, columns, index)
plt.imshow(Baseline_heatmap)
plt.title(f"Baseline")

plt.subplot(1, 4, 2) # (rows, columns, index)
plt.imshow(VICReg_heatmap)
plt.title(f"VICReg")

plt.subplot(1, 4, 3)
plt.imshow(MEMO_heatmap)
plt.title(f"MEMO")

plt.subplot(1, 4, 4) # (rows, columns, index)
plt.imshow(UP_Label_heatmap)
plt.title(f"UP-Label")

plt.subplots_adjust(wspace=0.02, hspace=0)

# Remove x and y ticks for all subplots
for ax in plt.gcf().get_axes():
    ax.set_xticks([])
    ax.set_yticks([])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Core No: 210, Inv: 30.00


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [121]:
core_info = test_core_ds.core_info.reset_index()
core_info[core_info["grade"] != 'Benign'].head(60)

,index,patient_id,id,loc,grade,pct_cancer,primary_grade,secondary_grade,tag,case_id,case_number,center,age,family_history,psa
13,38,3,38,RBL,GS7,50.0,3.0,4.0,CRCEO-0008_RBL,CRCEO-0008,8,CRCEO,57,True,4.00
17,52,5,52,LAL,GS7,3.0,3.0,4.0,CRCEO-0012_LAL,CRCEO-0012,12,CRCEO,69,True,18.02
19,57,5,57,RAM,GS7,35.0,3.0,4.0,CRCEO-0012_RAM,CRCEO-0012,12,CRCEO,69,True,18.02
20,58,5,58,RBL,GS7,10.0,3.0,4.0,CRCEO-0012_RBL,CRCEO-0012,12,CRCEO,69,True,18.02
21,60,5,60,RML,GS7,45.0,3.0,4.0,CRCEO-0012_RML,CRCEO-0012,12,CRCEO,69,True,18.02
22,61,5,61,RMM,GS7,25.0,3.0,4.0,CRCEO-0012_RMM,CRCEO-0012,12,CRCEO,69,True,18.02
25,76,7,76,LML,GS7,10.0,3.0,4.0,CRCEO-0017_LML,CRCEO-0017,17,CRCEO,62,False,3.40
26,77,7,77,LMM,GS7,40.0,3.0,4.0,CRCEO-0017_LMM,CRCEO-0017,17,CRCEO,62,False,3.40
27,78,7,78,RAL,GS7,40.0,4.0,3.0,CRCEO-0017_RAL,CRCEO-0017,17,CRCEO,62,False,3.40
42,120,12,120,LBL,GS7,20.0,4.0,3.0,CRCEO-0028_LBL,CRCEO-0028,28,CRCEO,71,False,6.50


In [36]:
core_info[core_info["grade"] == 'Benign'].head(50)

,index,patient_id,id,loc,grade,pct_cancer,primary_grade,secondary_grade,tag,case_id,case_number,center,age,family_history,psa
0,0,0,0,LAM,Benign,NaN,NaN,NaN,CRCEO-0004_LAM,CRCEO-0004,4,CRCEO,54,False,7.00
1,2,0,2,LML,Benign,NaN,NaN,NaN,CRCEO-0004_LML,CRCEO-0004,4,CRCEO,54,False,7.00
2,3,0,3,LMM,Benign,NaN,NaN,NaN,CRCEO-0004_LMM,CRCEO-0004,4,CRCEO,54,False,7.00
3,4,0,4,RBL,Benign,NaN,NaN,NaN,CRCEO-0004_RBL,CRCEO-0004,4,CRCEO,54,False,7.00
4,6,0,6,RML,Benign,NaN,NaN,NaN,CRCEO-0004_RML,CRCEO-0004,4,CRCEO,54,False,7.00
5,16,1,16,RBL,Benign,NaN,NaN,NaN,CRCEO-0005_RBL,CRCEO-0005,5,CRCEO,52,False,3.40
6,18,1,18,RML,Benign,NaN,NaN,NaN,CRCEO-0005_RML,CRCEO-0005,5,CRCEO,52,False,3.40
7,25,2,25,LMM,Benign,NaN,NaN,NaN,CRCEO-0006_LMM,CRCEO-0006,6,CRCEO,59,True,5.96
8,27,2,27,RAM,Benign,NaN,NaN,NaN,CRCEO-0006_RAM,CRCEO-0006,6,CRCEO,59,True,5.96
9,28,2,28,RBL,Benign,NaN,NaN,NaN,CRCEO-0006_RBL,CRCEO-0006,6,CRCEO,59,True,5.96


In [187]:
core_info.iloc[120]

index                         310
patient_id                     30
id                            310
loc                           RMM
grade                      Benign
pct_cancer                    NaN
primary_grade                 NaN
secondary_grade               NaN
tag                CRCEO-0070_RMM
case_id                CRCEO-0070
case_number                    70
center                      CRCEO
age                            69
family_history               True
psa                           9.3
Name: 120, dtype: object

In [ ]:
7